# Possum Dataset Analysis

## Dataset Context

This dataset is a classic practice regression dataset originally from the DAAG R package. It's used in examples and exercises in the book "Data Analysis and Graphics Using R" by Maindonald, J.H. and Braun, W.J. (2003, 2007, 2010). The dataset is also used in the OpenIntro Statistics book chapter 8: Introduction to Linear Regression.

The challenge is to use regression techniques to predict the age of a possum, its head length, and whether it is male or female.

## Dataset Content

The possum data frame consists of nine morphometric measurements on each of 104 mountain brushtail possums, trapped at seven sites from Southern Victoria to central Queensland.

## Acknowledgements

The data was originally found in the DAAG R package and used in the book "Data Analysis and Graphics Using R". A subset of the data was also put together for the OpenIntro Statistics book chapter 8: Introduction to Linear Regression.

## Original Source of Dataset

The original research was conducted by Lindenmayer, D. B., Viggers, K. L., Cunningham, R. B., and Donnelly, C. F. 1995. They focused on morphological variation among populations of the mountain brushtail possum, Trichosurus caninus Ogilby (Phalangeridae: Marsupiala). Their work was published in the Australian Journal of Zoology 43: 449-458.

## Research Questions

1. Can we use total length to predict a possum's head length?
2. Which possum body dimensions are most correlated with age and sex?
3. Can we classify a possum's sex by its body dimensions and location?
4. Can we predict a possum's trapping location from its body dimensions?

## Dataset Details

The dataset comes from the DAAG R package and contains nine morphometric measurements for each of 104 mountain brushtail possums, captured across seven locations from Southern Victoria to central Queensland.

## Educational Purpose

This dataset is commonly used for educational purposes, such as in the OpenIntro Statistics book, particularly in the chapter on linear regression.


In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("./possum.xls")

In [3]:
df.head()

,case,site,Pop,sex,age,hdlngth,skullw,totlngth,taill,footlgth,earconch,eye,chest,belly
0,1,1,Vic,m,8.0,94.1,60.4,89.0,36.0,74.5,54.5,15.2,28.0,36.0
1,2,1,Vic,f,6.0,92.5,57.6,91.5,36.5,72.5,51.2,16.0,28.5,33.0
2,3,1,Vic,f,6.0,94.0,60.0,95.5,39.0,75.4,51.9,15.5,30.0,34.0
3,4,1,Vic,f,6.0,93.2,57.1,92.0,38.0,76.1,52.2,15.2,28.0,34.0
4,5,1,Vic,f,2.0,91.5,56.3,85.5,36.0,71.0,53.2,15.1,28.5,33.0


In [4]:
cat_labels = [labels for labels, content in df.items(
) if pd.api.types.is_object_dtype(content)]

In [5]:
cat_labels

['Pop', 'sex']

In [6]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor

In [7]:
cat_encoder = Pipeline([
    ("ohe", OneHotEncoder(handle_unknown="ignore"))
])

In [8]:
preprocess = ColumnTransformer([
    ("cat_encoder", cat_encoder, cat_labels)
])

In [9]:
model = Pipeline([
    ("preprocess", preprocess),
    ("regr", XGBRegressor())
])

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X = df.drop("hdlngth", axis=1)
y = df["hdlngth"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [13]:
model.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('cat_encoder',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Pop', 'sex'])])),
                ('regr',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, device=None,
                              early_stopping_rounds=None,
                              enable_categorical=...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [14]:
from sklearn.metrics import mean_squared_error

In [15]:
y_preds = model.predict(X_test)

In [16]:
mean_squared_error(y_test, y_preds)

7.721656245746694

In [17]:
model.named_steps

{'preprocess': ColumnTransformer(transformers=[('cat_encoder',
                                  Pipeline(steps=[('ohe',
                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                  ['Pop', 'sex'])]),
 'regr': XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_t